In [2]:
import psycopg2
import json
import codecs

In [3]:
diagnosis_code = 'V420' # Kidney replaced by transplant
case_disease_name = 'Kidney_transplant'

# 1. derive case-control cohort 

In [3]:
def load_json_data(file_path, encoding='utf-8'):
    data = None
    with codecs.open(file_path, encoding=encoding) as rf:
        data = json.load(rf)
    return data


def query(conn_setting, sql):
    conn = psycopg2.connect(
            host=conn_setting['host'],
            database=conn_setting['database'],
            user=conn_setting['user'],
            password=conn_setting['password'])
    
    ret = None
    try:
        # create a cursor
        cur = conn.cursor()
        cur.execute(sql)
        ret = cur.fetchall()

        # close the communication with the PostgreSQL
        cur.close()
    finally:
        if conn is not None:
            conn.close()
        return ret
    

settings = load_json_data('./mimic_db_cnn.json')


## functions for retrieving patients by ICD-9 code or disease label

In [4]:

## get ICD 9 codes from disease label
def get_icd9_by_diag_label(disease):
    sql = """
    select icd9_code, long_title from mimiciii.d_icd_diagnoses where LOWER(long_title) like '%{disease}%' or short_title like '%{disease}%'
    """
    sql = sql.format(**{'disease': disease})
    disease_codes_ret = query(settings, sql)

    disease_codes = [t[0] for t in disease_codes_ret]
    print('disease icd9 codes', disease_codes)
    return disease_codes

## get patient stats (grouped by icd9)
def get_patients_stats_by_icd9(codes):
    sql = """
    select count(*) num, dd.long_title from mimiciii.diagnoses_icd d, mimiciii.d_icd_diagnoses dd 
    where d.icd9_code in (%s) 
        and dd.icd9_code=d.icd9_code group by dd.long_title order by num desc
    """

    sql = sql % ','.join(["'%s'" % s for s in codes])
    case_code_stats = query(settings, sql)
    print('case_code_stats', case_code_stats)
    

## get patients' subject id and admission_id lists by icd code
def get_patients_by_icd9(codes, other_conditions=None):
    sql = """
    select distinct subject_id, hadm_id from mimiciii.diagnoses_icd d
    where d.icd9_code in ({icd_codes})
    {other_conditions}
    """
    s_other_conditions = ''
    if other_conditions is not None:
        s_other_conditions = ' and '.join(other_conditions)
    sql = sql.format(**{'icd_codes': ','.join(["'%s'" % s for s in codes]),
                        'other_conditions': s_other_conditions
                        })
    disease_patients = query(settings, sql)
    print('distinct admissions', len(disease_patients))
    return disease_patients

## get subject-hadm ids for cases
- `subject_id` is the patient id 
- `hadm_id` is the unique admission id

In [5]:
# get case icd codes
# case_codes = get_icd9_by_diag_label(case_disease_name)
case_codes = [diagnosis_code]

other_conditions = ["ethnicity = "]

# get patient stats by individual icd9 code
get_patients_stats_by_icd9(case_codes)

# get case patient ids
case_patients = get_patients_by_icd9(case_codes)

case_code_stats [(337, 'Kidney replaced by transplant')]
distinct admissions 337


## demographic info of patients

In [6]:
import pandas as pd

sql = """
select 
    a.subject_id, a.hadm_id, a.admittime, p.dob, diagnosis,
    a.ethnicity,
    date_part('year', age(a.admittime, p.dob)), 
    p.gender, 
    date_part('day', age(a.dischtime, a.admittime)),
    date_part('day', age(a.deathtime, a.admittime)) 
    
    from mimiciii.admissions a, mimiciii.patients p where a.subject_id=p.subject_id 
    and a.subject_id || ',' || a.hadm_id in (%s)
"""
sql = sql % ','.join(["'%s,%s'" % (t[0], t[1]) for t in case_patients])
cohort = query(settings, sql)


In [11]:
df_cohort = pd.DataFrame(cohort, columns=['patient_id', 'admission_id', 
                                          'adm_time', 'DoB', 'diag', 'ethnicity',
                                          'age', 'gender', 'LoS', 'Adm2Dth'])
df_cohort.head(100)

,patient_id,admission_id,adm_time,DoB,diag,ethnicity,age,gender,LoS,Adm2Dth
0,361,148959,2122-01-05 22:58:00,2067-11-18,DEEP VEIN THROMBOSIS;HEMOCULT POSITIVE,MULTI RACE ETHNICITY,54.0,M,18.0,NaN
1,323,192631,2115-05-17 21:06:00,2062-12-24,ACUTE MI,WHITE,52.0,M,9.0,NaN
2,323,106158,2116-05-03 23:07:00,2062-12-24,CONGESTIVE HEART FAILURE,WHITE,53.0,M,7.0,NaN
3,323,143334,2120-01-11 15:46:00,2062-12-24,SEPSIS;TELEMETRY,WHITE,57.0,M,6.0,NaN
4,327,146750,2194-04-28 09:15:00,2152-03-25,MALUNION FRACTURE RIGHT WRIST/SDA,WHITE,42.0,F,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
95,11003,163654,2123-03-01 23:30:00,2067-09-29,FEVER,WHITE,55.0,M,8.0,NaN
96,9384,187655,2106-11-07 19:52:00,2046-11-22,CHEST PAIT;+ETT\CATH,WHITE,59.0,M,7.0,NaN
97,11338,109986,2122-07-12 07:33:00,2077-07-01,INSULIN DEPENDENT DIABETES,WHITE,45.0,M,12.0,NaN
98,11472,142545,2150-02-17 12:58:00,2101-02-02,MI/S/P TRANSPLANT\CATH,WHITE,49.0,M,6.0,NaN


In [12]:
df_cohort['case'] = 1
df_cohort.describe()

,patient_id,admission_id,age,LoS,Adm2Dth,case
count,337.000000,337.000000,337.000000,337.000000,24.000000,337.0
mean,30137.089021,150473.246291,54.477745,8.504451,9.125000,1.0
std,27277.902364,29159.005737,18.262152,6.141739,8.082792,0.0
min,323.000000,100229.000000,17.000000,0.000000,0.000000,1.0
25%,9384.000000,124081.000000,46.000000,4.000000,0.750000,1.0
50%,21015.000000,149765.000000,54.000000,7.000000,10.000000,1.0
75%,48327.000000,175626.000000,62.000000,11.000000,15.000000,1.0
max,99650.000000,199955.000000,302.000000,28.000000,23.000000,1.0


In [13]:
## inclusion criteria
df_cohort.drop(df_cohort[df_cohort['age'] < 18].index, inplace=True)

## match control cohort

In [14]:
sql = """
select 
    a.subject_id, a.hadm_id, a.diagnosis,
    date_part('year', age(a.admittime, p.dob)) 
    from mimiciii.admissions a, mimiciii.patients p
    where a.subject_id=p.subject_id 
    and a.subject_id || ',' || a.hadm_id not in (%s)
"""
sql = sql % ','.join(["'%s,%s'" % (t[0], t[1]) for t in case_patients])
controls = query(settings, sql)
print(len(controls))

58639


In [15]:
import random

# matching usnig diagnosis and similar age
def match_cohort(r, d2c, k, already_matched):
    matched = []
    if r['diag'] not in d2c:
        return matched
    for c in d2c[r['diag']]:
        if abs(r['age'] - c[3]) <=5 and c not in already_matched:
            matched.append(c)
    random.shuffle(matched)
    return matched[:k]

# matching only with age similarity
def match_cohort_by_age(r, controls, k, already_matched):
    matched = []
    for c in controls:
        if abs(r['age'] - c[3]) <=2 and c not in already_matched:
            if len(matched) > 3 * k:
                break
            matched.append(c)
    random.shuffle(matched)
    return matched[:k]


d2c = {}
for c in controls:
    if c[2] not in d2c:
        d2c[c[2]] = []
    d2c[c[2]].append(c)
    
print('diag to control dict generated')
    
k = 4
not_matched = 0

matched_controls = []
for _, r in df_cohort.iterrows():
    matched = match_cohort(r, d2c, k, matched_controls)
#     matched = match_cohort_by_age(r, controls, k, matched_controls)
    if len(matched) == 0:
        not_matched += 1
    if len(matched) > 0:
        matched_controls += matched

print('not_matched', not_matched)
print('total matched control', len(matched_controls))
print('sample', matched_controls[:3])

diag to control dict generated
not_matched 155
total matched control 654
sample [(14640, 152772, 'ACUTE MI', 57.0), (13192, 175809, 'ACUTE MI', 49.0), (8906, 194915, 'ACUTE MI', 55.0)]


## load matched cohort to the dataframe

In [16]:
sql = """
select 
    a.subject_id, a.hadm_id, a.admittime, p.dob, diagnosis,
    a.ethnicity,
    date_part('year', age(a.admittime, p.dob)), 
    p.gender, 
    date_part('day', age(a.dischtime, a.admittime)),
    date_part('day', age(a.deathtime, a.admittime)) 
    
    from mimiciii.admissions a, mimiciii.patients p where a.subject_id=p.subject_id 
    and a.subject_id || ',' || a.hadm_id in (%s)
"""
sql = sql % ','.join(["'%s,%s'" % (t[0], t[1]) for t in matched_controls])
matched_control_data = query(settings, sql)
print('loaded data size', len(matched_control_data))

loaded data size 654


## serialisation point: save cohort to csv file

In [18]:
# merge matched control to the cohort dataframe
df_control = pd.DataFrame(matched_control_data, columns=df_cohort.columns[:10])
df_control['case'] = 0
df_cohort = df_cohort.append(df_control)

# save csv
csv_file = './%s_case_control.csv' % case_disease_name
df_cohort.to_csv(csv_file, index=False)
print('saved')

saved


# 2. load clinical attributes for prediction

In [40]:
df_cohort.head(2)

,patient_id,admission_id,adm_time,DoB,diag,ethnicity,age,gender,LoS,Adm2Dth,case
0,361,148959,2122-01-05 22:58:00,2067-11-18,DEEP VEIN THROMBOSIS;HEMOCULT POSITIVE,MULTI RACE ETHNICITY,54.0,M,18.0,NaN,1
1,323,192631,2115-05-17 21:06:00,2062-12-24,ACUTE MI,WHITE,52.0,M,9.0,NaN,1


- Chronic heart failure codes using `428%`
- leukemia using label matched icd9 codes
- cirrhosis using label matched icd9 codes

In [83]:
def binary_feature_query(icd_codes, patient_ids, like=False):
    sql = """
    select count(*), subject_id from mimiciii.diagnoses_icd d where d.icd9_code {icd_code_pattern} and d.subject_id in ({patient_ids})
    group by subject_id
    """ 
    if like:
        icd_code_pattern = "like '%s'" % icd_codes
    else:
        icd_code_pattern = 'in (%s)' %(','.join(["'%s'" % d for d in icd_codes]))
    sql = sql.format(**{'icd_code_pattern': icd_code_pattern,
                        'patient_ids': ','.join(["'%s'" % d for d in patient_ids])
                       })
    return query(settings, sql)

def add_feature_by_query(df, icd_codes, feature_name, index_field='patient_id', like=False):
    unique_patients = list(set(df[index_field].tolist()))
    print('unique patient #', len(unique_patients))
    features = binary_feature_query(icd_codes, unique_patients, like=like)
    df_features = pd.DataFrame(features, columns=[feature_name, index_field])
    df_new = df.join(df_features.set_index(index_field), on=index_field, how='left')
    df_new[feature_name] = df_new[feature_name].apply(lambda v: 0 if pd.isnull(v) else v)
    return df_new

In [98]:
chf_codes = '428%'
df_new = add_feature_by_query(df_cohort, chf_codes, 'CHF', like=True)

unique patient # 861


## add leukemia feature

In [99]:
leukemia_label = 'leukemia'
leukemia_codes = get_icd9_by_diag_label(leukemia_label)
df_new = add_feature_by_query(df_new, leukemia_codes, 'Leukemia')

disease icd9 codes ['20310', '20311', '20312', '20400', '20401', '20402', '20410', '20411', '20412', '20420', '20421', '20422', '20480', '20481', '20482', '20490', '20491', '20492', '20500', '20501', '20502', '20510', '20511', '20512', '20520', '20521', '20522', '20580', '20581', '20582', '20590', '20591', '20592', '20600', '20601', '20602', '20610', '20611', '20612', '20620', '20621', '20622', '20680', '20681', '20682', '20690', '20691', '20692', '20700', '20701', '20702', '20720', '20721', '20722', '20780', '20781', '20782', '20800', '20801', '20802', '20810', '20811', '20812', '20820', '20821', '20822', '20880', '20881', '20882', '20890', '20891', '20892', 'V1060', 'V1061', 'V1062', 'V1063', 'V1069', 'V166']
unique patient # 861


In [101]:
df_new['Leukemia'].value_counts()

0.0    979
1.0      7
3.0      2
2.0      1
6.0      1
Name: Leukemia, dtype: int64

In [102]:
cirrhosis_codes = get_icd9_by_diag_label('cirrhosis')
df_new = add_feature_by_query(df_new, cirrhosis_codes, 'cirrhosis')

disease icd9 codes ['5712', '5715', '5716']
unique patient # 861


In [104]:
df_new['cirrhosis'].value_counts()

0.0     908
1.0      46
2.0      16
3.0       7
4.0       6
5.0       5
10.0      1
6.0       1
Name: cirrhosis, dtype: int64

# 3. generate train-dev-test dataframe

In [ ]:
from sklearn.model_selection import train_test_split

df_cohort = pd.read_csv(csv_file)
X = df_cohort[['patient_id', 'row_id']].apply(lambda r: '%s_%s' % (r[0], r[1]), axis=1)
y = df_cohort['case']

X_train_dev, X_test, y_train_dev, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=1)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev, y_train_dev, test_size=0.1, stratify=y_train_dev, shuffle=True, random_state=1)

In [ ]:
train_csv = './sr_%s_train.csv' % case_disease_name
dev_csv = './sr_%s_dev.csv' % case_disease_name
test_csv = './sr_%s_test.csv' % case_disease_name
pd.DataFrame({'sr_id': X_train, case_disease_name: y_train}).to_csv(train_csv, index=False)
pd.DataFrame({'sr_id': X_dev, case_disease_name: y_dev}).to_csv(dev_csv, index=False)
pd.DataFrame({'sr_id': X_test, case_disease_name: y_test}).to_csv(test_csv, index=False)
print('training/dev/testing saved')